# Multi-Table Database - Tracks
#### All cells must be executed in order to have the result

In [139]:
''' SFM - Nov, 2021
    This is the result of the specialization task, 
    it is expected a xml exported from itunes or a source that contains the itunes exported file    
'''
import sqlite3
import xml.etree.ElementTree as ET
from urllib.request import urlopen

tree = None
localXmlFile = 'Library.xml'
fileHandle = None

## Connecting with the Database

In [140]:
#connect to database
db = sqlite3.connect('Multi-Table-Database-Tracks.sqlite')
#create the cursor
cursor = db.cursor()

#clean up the previous data
try:
    print('clearing up the database')
    cursor.executescript('''
    DROP TABLE IF EXISTS Artist;
    DROP TABLE IF EXISTS Album;
    DROP TABLE IF EXISTS Track;
    DROP TABLE IF EXISTS Genre;
    
    CREATE TABLE Artist (
        id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name    TEXT UNIQUE
    );

    CREATE TABLE Genre (
        id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name    TEXT UNIQUE
    );

    CREATE TABLE Album (
        id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        artist_id  INTEGER,
        title   TEXT UNIQUE
    );

    CREATE TABLE Track (
        id  INTEGER NOT NULL PRIMARY KEY 
            AUTOINCREMENT UNIQUE,
        title TEXT  UNIQUE,
        album_id  INTEGER,
        genre_id  INTEGER,
        len INTEGER, rating INTEGER, count INTEGER
    );
    ''')
    print('clearing up action successfully finished')
except:
    print('error while clearing up the database')


clearing up the database
clearing up action successfully finished


## Defs

### Get key and value of each track

In [141]:
#This funtion will return the key, value result of each parameter to be inserted to dictionary             
def getKeyValue(currentTrack):
    trackAttributes = dict()
    #newTrackKeyFound = False #When True that means that the code found a 'Track Id' tag
    key = None #Is the attribute name on xml
    value = None #Is the value of the atribute on xml
    newValue = False #If the value was found, change to true 
    newKey = False #If the key tag was found, change to true
    
    for attribute in currentTrack:
        #get the key and value for the attributes
        if attribute.tag == 'key': #not newTrackKeyFound and
            key = attribute.text
            newKey = True
            #print('\t', key)
        elif  attribute.tag != 'key': #not newTrackKeyFound and
            value = attribute.text
            newValue = True
            #print('\t', value)

        #print (newKey, ' - ', newValue)
        if newKey and newValue:            
            #put the value in the dictionary 
            key = key.replace(' ', '')
            #print('\t {}:{}'.format(key, value))
            trackAttributes[key] = value
            
            #reset the values
            key = None 
            value = None 
            newValue = False
            newKey = False
    return trackAttributes

### Checking the dictionary key and adding into database

In [142]:
def insertTrackIntoDatabase(currentTrack, cursor):
    #check if the values that are going to be used to sql exists on dict, listed are the 'Main' values
    values = 'Artist', 'Album', 'Name', 'Genre'
    for attribute in values: 
        if attribute not in (currentTrack.keys()):
            print('Value {} does not exist for track id: {} . Track will not be added to database'.format(attribute, currentTrack['TrackID']))
            return
    
    #the IGNORE works for sqlite and avoid duplicate addition of the attribute    
    insertArtist = 'INSERT OR IGNORE INTO Artist(NAME) VALUES (:Artist)'
    insertAlbum = 'INSERT OR IGNORE INTO Album(title, artist_id) VALUES(:Album, :ArtistId)'
    insertTrack = '''INSERT OR IGNORE INTO Track(title, album_id, len, rating, count, genre_id) 
                                                VALUES(:Name, :AlbumId, :TotalTime, :Rating, :PlayCount,:GenreId)'''
    insertGenre = 'INSERT OR IGNORE INTO Genre(NAME) VALUES (:Genre)'
    
    #select clauses 
    selectArtistId = "SELECT id FROM Artist WHERE NAME = :Artist;" 
    selectAlbumId = "SELECT id FROM Album WHERE Title = :Album"
    selectGenreId = "SELECT id FROM Genre WHERE Name = :Genre"
    
    #First Insert the Artist
    cursor.execute(insertArtist, currentTrack)
    
    #Then get the id of the artist, as it is random, this insert is needed
    cursor.execute(selectArtistId, currentTrack)#WHERE name = ':Artist'", currentTrack)
    
    # add the information to current dict and use it in Album insert
    artistId = cursor.fetchone()[0]
    currentTrack['ArtistId'] = artistId
    
    #Add the Album
    cursor.execute(insertAlbum, currentTrack)
    
    #get the Album Id
    cursor.execute(selectAlbumId, currentTrack)
    albumId = cursor.fetchone()[0]
    currentTrack['AlbumId'] = albumId
    
    #Add the Genre
    cursor.execute(insertGenre, currentTrack)
    
    #get the Genre Id
    cursor.execute(selectGenreId, currentTrack)
    genreId = cursor.fetchone()[0]
    print('genreid', genreId)
    currentTrack['GenreId'] = genreId
    
    #check if the fields to be added in this table exists in dictionary, this same action is perfomed above for other fields
    #but in this case, the record can be inserted, and these fields, if no on dictionary will be created with null value
    trackFields = 'TotalTime','Rating','PlayCount'
    for attribute in trackFields: 
        if attribute not in (currentTrack.keys()):
            currentTrack[attribute] = None
   
    #And then Add the track
    cursor.execute(insertTrack, currentTrack)

## Main Execution

In [143]:
#requesting from the user the xml through url
url = input('Enter itunes playlist xml: ')

#If user provides a url, access the url and get the file data
if len(url) < 1:
    try:
        #open the local file
        print('Getting Local File', localXmlFile)
        fileHandle = open(localXmlFile)
        tree = ET.parse(fileHandle)
    except Exception as e: 
        print('It is not possible to open the local file', localXmlFile)
else: 
    try:
        #open url
        fileHandle = urlopen(url).read()
        tree = ET.fromstring(fileHandle)
    except Exception as e: 
        print(e)
        exit()
#Parse the data into the tree
    
trackList = tree.findall('dict/dict/dict')
print(len(trackList))
for currentTrack in trackList:
    trackDict = getKeyValue(currentTrack)
    #print(trackDict)
    insertTrackIntoDatabase(trackDict, cursor)
db.commit()

Enter itunes playlist xml: 
Getting Local File Library.xml
404
genreid 1
genreid 2
genreid 3
genreid 1
genreid 1
genreid 6
genreid 1
genreid 1
genreid 9
genreid 1
genreid 1
genreid 1
genreid 3
genreid 14
genreid 1
Value Album does not exist for track id: 399 . Track will not be added to database
genreid 3
genreid 9
genreid 1
genreid 1
genreid 14
genreid 9
genreid 1
genreid 23
genreid 14
genreid 9
genreid 1
genreid 2
genreid 14
genreid 1
genreid 14
genreid 23
genreid 1
genreid 9
genreid 9
genreid 14
genreid 2
genreid 9
genreid 1
genreid 23
genreid 3
genreid 1
genreid 23
genreid 1
genreid 1
genreid 23
genreid 3
genreid 23
genreid 23
genreid 1
genreid 14
genreid 14
genreid 14
genreid 23
genreid 23
genreid 14
genreid 14
genreid 14
genreid 14
genreid 14
genreid 14
genreid 14
Value Album does not exist for track id: 493 . Track will not be added to database
genreid 14
genreid 9
genreid 1
genreid 1
genreid 23
genreid 1
genreid 3
genreid 14
genreid 1
genreid 1
genreid 3
genreid 1
genreid 1
gen